In [1]:
import uuid
from typing import TypedDict, List, Optional
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# --- 1. SETUP MODEL ---
# Using a higher temperature (0.7) so the motivation isn't repetitive
llm = ChatOllama(model="qwen2.5:0.5b", temperature=0.7)

# --- 2. DEFINE STATE ---
class HabitState(TypedDict):
    # Inputs
    user_name: str
    activity: str           # e.g., "Gym"
    
    # Long-term Memory (Persisted)
    streak_count: int
    history: List[str]      # Log of past entries
    
    # Outputs
    motivational_message: str

# --- 3. DEFINE NODES ---

def tracker_node(state: HabitState):
    """
    Updates the streak and logs the activity.
    """
    activity = state["activity"]
    # Get current streak from state (defaults to 0 if new)
    current_streak = state.get("streak_count", 0)
    history = state.get("history", [])
    
    # Logic: Increment streak
    new_streak = current_streak + 1
    new_history = history + [f"Completed {activity} (Streak: {new_streak})"]
    
    print(f"📈 Updating Stats: Streak is now {new_streak}!")
    
    # We return the UPDATED keys. LangGraph merges this into the state.
    return {
        "streak_count": new_streak,
        "history": new_history
    }

def motivator_node(state: HabitState):
    """
    Generates a custom message based on the streak.
    """
    name = state["user_name"]
    streak = state["streak_count"]
    activity = state["activity"]
    
    # Dynamic context based on streak length
    context = ""
    if streak == 1:
        context = "This is their first day. Be encouraging but gentle."
    elif streak < 5:
        context = "They are building momentum. Tell them to keep it up!"
    else:
        context = "They are on fire! Be hyped and energetic! Use emojis."
        
    prompt = ChatPromptTemplate.from_template(
        """You are a high-energy fitness coach.
        User: {name}
        Activity: {activity}
        Current Streak: {streak} days.
        Context: {context}
        
        Write a short, punchy motivational message (max 1 sentence) for them.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "name": name, 
        "activity": activity, 
        "streak": streak,
        "context": context
    })
    
    return {"motivational_message": response.content}

# --- 4. BUILD GRAPH ---

workflow = StateGraph(HabitState)

workflow.add_node("tracker", tracker_node)
workflow.add_node("motivator", motivator_node)

workflow.set_entry_point("tracker")
workflow.add_edge("tracker", "motivator")
workflow.add_edge("motivator", END)

# *** CRITICAL: PERSISTENCE ***
# This object saves the state in memory using the thread_id
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# --- 5. RUN IT (SIMULATING DAYS) ---

# We use a specific thread_id to represent "Himanshu's Profile"
# If we used a different ID, it would treat you as a new user with 0 streak.
thread_config = {"configurable": {"thread_id": "user_himanshu_123"}}

print("\n--- 📅 DAY 1: Going to the Gym ---")
# Initial input needs to set the baselines (or handle them in code)
inputs = {"user_name": "Himanshu", "activity": "Gym", "streak_count": 0, "history": []}

result1 = app.invoke(inputs, config=thread_config)
print(f"💪 Message: {result1['motivational_message']}")

print("\n\n... Sleeping for 24 hours (Simulated) ...\n")

print("--- 📅 DAY 2: Coding ---")
# Notice: We DO NOT pass 'streak_count' here! The graph remembers it from Day 1.
inputs_day2 = {"user_name": "Himanshu", "activity": "Coding"}

result2 = app.invoke(inputs_day2, config=thread_config)
print(f"Current Streak: {result2['streak_count']}")
print(f"💻 Message: {result2['motivational_message']}")

print("\n\n... Another day passes ...\n")

print("--- 📅 DAY 3: Reading ---")
inputs_day3 = {"user_name": "Himanshu", "activity": "Reading"}

result3 = app.invoke(inputs_day3, config=thread_config)
print(f"Current Streak: {result3['streak_count']}")
print(f"📖 Message: {result3['motivational_message']}")


--- 📅 DAY 1: Going to the Gym ---
📈 Updating Stats: Streak is now 1!
💪 Message: "Every step forward counts! Remember to make every movement count and you'll see incredible progress soon!"


... Sleeping for 24 hours (Simulated) ...

--- 📅 DAY 2: Coding ---
📈 Updating Stats: Streak is now 2!
Current Streak: 2
💻 Message: Congratulations on your coding journey! Keep pushing yourself and stay inspired! Remember, each step forward is a victory. Keep going, Himanshu!


... Another day passes ...

--- 📅 DAY 3: Reading ---
📈 Updating Stats: Streak is now 3!
Current Streak: 3
📖 Message: Here's the short, punchy motivational message you can use with Himanshu:

"Keep pushing forward! Every step counts and every day is an opportunity!"
